<a target="_blank" href="https://colab.research.google.com/github/alexmelekhin/cv_course_2023/blob/main/seminars/seminar_14/place_recognition.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Задание: запуск и обучение MinkLoc++

В данном задании Вам предлагается обучить модель MinkLoc++ на датасете Oxford RobotCar, используя библиотеку OpenPlaceRecognition.

_Это задание опционально и не будет оцениваться._

## Предварительная настройка

### Установка MinkowskiEngine в Google Colab

При желании установить локально или где-либо еще - обращайтесь к инструкциям в репозитории: https://github.com/NVIDIA/MinkowskiEngine 

In [ ]:
# вам потребуется GPU, проверьте ее наличие
!nvidia-smi

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1

In [ ]:
!pip install ninja

In [ ]:
import torch
print(f"torch version: {torch.__version__}")
print(f"Is CUDA available in torch?: {torch.cuda.is_available()}")

In [ ]:
!sudo apt-get install libopenblas-dev

In [ ]:
!pip install -U git+https://github.com/NVIDIA/MinkowskiEngine -v --no-deps \
                          --global-option="--force_cuda" \
                          --global-option="--blas=openblas"

In [ ]:
import torch
print(f"Is CUDA available in torch?: {torch.cuda.is_available()}")
import MinkowskiEngine as ME
print(f"Is CUDA available in MinkowskiEngine?: {ME.is_cuda_available()}")
ME.print_diagnostics()

### Загрузка OpenPlaceRecognition

In [ ]:
!git clone https://github.com/alexmelekhin/OpenPlaceRecognition.git
%cd OpenPlaceRecognition
!git checkout dev  # ! важно, в main может быть неактуальный код
!pip install -e .

### Загрузка датасета в Google Colab

Вы можете воспользоваться утилитой gdown, которая по умолчанию доступна в Colab. Допустим, https://drive.google.com/file/d/1b2ry0PGa3vnl8gVhEWqRz329y_ekZX9C/view?usp=share_link - ссылка на файл. Чтобы скачать его, нам нужно передать в gdown в качестве аргумента его id - для данного примера это `1b2ry0PGa3vnl8gVhEWqRz329y_ekZX9C` (часть ссылки между `file/d/` и `/view`).

In [ ]:
!gdown 1b2ry0PGa3vnl8gVhEWqRz329y_ekZX9C

Распаковка архива:

In [ ]:
!unzip -q pnvlad_oxford_robotcar.zip

**Важно:** архив с датасетом слегка устарел, дополнительно необходимо загрузить обновленные файлы train.csv, val.csv и test.csv отсюда: https://drive.google.com/drive/folders/1_KJQ8350zO0lCGfk4_BkjHMUa5RWnK63?usp=sharing 

Замените старые файлы из директории pnvlad_oxford_robotcar на новые.

## Запуск обучения

Перед тем, как запускать обучение с помощью скрипта рекомендуется внимательно проверить конфиг в файле `OpenPlaceRecognition/configs/config.yaml`. Проверьте, что значения конфига установлены следующим образом:

```
defaults:
  - _self_
  - model: minkloc_multimodal
  - dataset: oxford
  - loss: triplet_margin
  - optimizer: adam
  - scheduler: multi_step

general:
  debug: False
  seed: 31299
  checkpoints_dir: checkpoints/
  device: cuda
  num_workers: 4
  batch_expansion_th: 0.7
  modalities: [image, cloud,]
  test_modality: fusion
  epochs: 60
```

In [ ]:
# проверьте, в какой вы директории сейчас
!pwd

In [ ]:
# перейдите в директорию с репозиторием, если еще не
%cd OpenPlaceRecognition

Обучение запускается скриптом `train.py`. При необходимости, можно переопределить некоторые значения конфига:

In [ ]:
!python train.py wandb.disabled=True dataset.dataset.dataset_root=/content/pnvlad_oxford_robotcar